In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display

import requests

import psycopg2
from netrc import netrc
from fiona.crs import from_epsg

user, acct, passwd = netrc().authenticators("harris")

%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10000

fips, epsg = 22, 3552
laf = "/media/jsaxon/brobdingnag/data/el_dv/la_site/"

In [2]:
! ls /media/jsaxon/brobdingnag/data/el_dv/la_site/

2012_LA_Precincts.dbf		       LA_2016.cpg	    la_geo_votes_16.shp
2012_LA_Precincts.prj		       LA_2016.dbf	    la_geo_votes_16.shx
2012_LA_Precincts.sbn		       LA_2016.prj	    la_geo_votes.cpg
2012_LA_Precincts.sbx		       LA_2016.shp	    la_geo_votes.dbf
2012_LA_Precincts.shp		       LA_2016.shx	    la_geo_votes.prj
2012_LA_Precincts.shp.xml	       la_geo_votes_12.cpg  la_geo_votes.shp
2012_LA_Precincts.shx		       la_geo_votes_12.dbf  la_geo_votes.shx
2012 Louisiana Precinct Shapefile.zip  la_geo_votes_12.prj  la.qgs
2016_LA_PRECINCTS.DBF		       la_geo_votes_12.shp  la_votes_2012.csv
2016_LA_Precincts.prj		       la_geo_votes_12.shx  la_votes_2016.csv
2016_LA_Precincts.shp		       la_geo_votes_16.cpg  sources.txt
2016_LA_Precincts.shx		       la_geo_votes_16.dbf
2016_LouisianaPrecinctShapefile.zip    la_geo_votes_16.prj


In [3]:
! cat /media/jsaxon/brobdingnag/data/el_dv/la_site/sources.txt

https://voterportal.sos.la.gov/static/#/
https://voterportal.sos.la.gov/static/#/2016-11-08/resultsRegion/53898/01

>> Get JSON here, by county (1-64)
https://voterportal.sos.la.gov/ElectionResults/ElectionResults/Data?blob=20161108/VotesRaceByPrecinct/Votes_53898_01.htm

http://house.louisiana.gov/H_Redistricting2011/default_LouisianaPrecinctShapefiles.htm


In [4]:
addr12 = "https://voterportal.sos.la.gov/ElectionResults/ElectionResults/Data?blob=20121106/VotesRaceByPrecinct/Votes_46257_{:02d}.htm"

df12 = []
for ci in []: # range(1, 65):

    js = requests.get(addr12.format(ci)).json()

    data = js["Precincts"]["Precinct"]
    for precinct in data:
        for choice in precinct["Choice"]:
            precinct[int(choice["ID"])] = int(choice["VoteTotal"])

    df = pd.DataFrame(data)
    df = df.rename(columns = {"Precinct" : "precinct", 90532 : "R12", 90536 : "D12"})
    df = df[["precinct", "D12", "R12"]]
    df["county"] = ci*2-1
    df12.append(df)
    
# df12 = pd.concat(df12)
# df12.replace({"precinct" : {"Early Voting" : "EV", "Provisional Votes" : "PR"}}, inplace = True)
# df12[["county", "precinct", "D12", "R12"]].to_csv(laf + "la_votes_2012.csv", index = False)

In [5]:
addr16 = "https://voterportal.sos.la.gov/ElectionResults/ElectionResults/Data?blob=20161108/VotesRaceByPrecinct/Votes_53898_{:02d}.htm"

df16 = []
for ci in []: # range(1, 65):

    js = requests.get(addr16.format(ci)).json()

    data = js["Precincts"]["Precinct"]
    for precinct in data:
        for choice in precinct["Choice"]:
            # print(choice)
            precinct[int(choice["ID"])] = int(choice["VoteTotal"])

    df = pd.DataFrame(data)
    df = df.rename(columns = {"Precinct" : "precinct", 101869 : "R16", 101870 : "D16"})
    df = df[["precinct", "D16", "R16"]]
    df["county"] = ci*2-1
    df16.append(df)
    
# df16 = pd.concat(df16)
# df16.replace({"precinct" : {"Early Voting" : "EV", "Provisional Votes" : "PR"}}, inplace = True)
# df16[["county", "precinct", "D16", "R16"]].to_csv(laf + "la_votes_2016.csv", index = False)

In [6]:
votes_12 = pd.read_csv(laf + "la_votes_2012.csv")
votes_16 = pd.read_csv(laf + "la_votes_2016.csv")

In [7]:
def allocate_prov_abs_votes(votes, yr):
    
    D, R = "D{}".format(yr%100), "R{}".format(yr%100)

    poll_mask = ~votes.precinct.str.contains("EV") & ~votes.precinct.str.contains("PR")
    
    poll_totals = votes[poll_mask].groupby("county").sum()[[D, R]].reset_index()\
                                  .rename(columns = {D : D + "_poll_tot", R : R + "_poll_tot"})\

    alt_totals = votes[~poll_mask].groupby("county").sum()[[D, R]].reset_index()\
                                  .rename(columns = {D : D + "_alt_tot", R : R + "_alt_tot"})\
                                          
    votes = pd.merge(votes, poll_totals, on = "county", how = "outer")
    votes = pd.merge(votes, alt_totals,  on = "county", how = "outer")
    votes.fillna(0, inplace = True)

    votes[D + "_poll_frac"], votes[R + "_poll_frac"] = 0, 0
    votes.loc[poll_mask, D + "_poll_frac"] = 1. * votes.loc[poll_mask, D] / votes.loc[poll_mask, D + "_poll_tot"]
    votes.loc[poll_mask, R + "_poll_frac"] = 1. * votes.loc[poll_mask, R] / votes.loc[poll_mask, R + "_poll_tot"]

    votes.loc[poll_mask, D] += votes.loc[poll_mask, D + "_alt_tot"] * votes.loc[poll_mask, D + "_poll_frac"]
    votes.loc[poll_mask, R] += votes.loc[poll_mask, R + "_alt_tot"] * votes.loc[poll_mask, R + "_poll_frac"]
        
    votes = votes.loc[poll_mask, ["county", "precinct", D, R]]
            
    return votes


### Gruesome merging for 2012

In [8]:
votes_12 = pd.read_csv(laf + "la_votes_2012.csv")

votes_12.loc[votes_12.county.isin([5, 7, 17, 33, 55, 57, 87, 89, 97, 109]), "precinct"] = votes_12.loc[votes_12.county.isin([5, 7, 17, 33, 55, 57, 87, 89, 97, 109]), "precinct"].str.replace(r" [ABCYZJKLM]", "")
votes_12.loc[votes_12.county.isin([15]), "precinct"] = votes_12.loc[votes_12.county.isin([15]), "precinct"].str.replace(r" -*[1-2]$", r"")
votes_12.loc[votes_12.county.isin([25, 43, 75, 89, 91, 99, 127]), "precinct"] = votes_12.loc[votes_12.county.isin([25, 43, 75, 89, 91, 99, 127]), "precinct"].str.replace(r"/", r"/0")
votes_12.loc[votes_12.county.isin([29]), "precinct"] = votes_12.loc[votes_12.county.isin([29]), "precinct"].str.replace(r"00/", r"0").str.replace(r"-", r"/0").str.replace(r"[24][AB]", "2")
votes_12.loc[votes_12.county.isin([67]), "precinct"] = votes_12.loc[votes_12.county.isin([67]), "precinct"].str.replace(r"00/([0-9]{2})-([12])", r"\1/0\2")
votes_12.loc[votes_12.county.isin([79]), "precinct"] = votes_12.loc[votes_12.county.isin([79]), "precinct"].str.replace(r" [AL][KZ]", "")

votes_12.loc[(votes_12.county == 3)   & (votes_12.precinct == "02/02A"), "precinct"] = "02/02-A"
votes_12.loc[(votes_12.county == 113) & (votes_12.precinct == "00/49B-2"), "precinct"] = "00/49B-1"
votes_12.loc[(votes_12.county == 121) & (votes_12.precinct == "00/02A"), "precinct"] = "00/02B"
votes_12.loc[(votes_12.county == 121) & (votes_12.precinct == "00/11B"), "precinct"] = "00/11A"

# votes_12.loc[votes_12.county == 105, "precinct"] = votes_12.loc[votes_12.county == 105, "precinct"].str.replace(r"0(28|40|42|45|70|72)A", r"0\1")


votes_12 = allocate_prov_abs_votes(votes_12, 2012)
votes_12 = votes_12.groupby(by = ["county", "precinct"]).sum().reset_index()

In [9]:
geo_12 = gpd.read_file(laf + "2012_LA_Precincts.shp") # Had to re-save on the Mac to get field data.
geo_12["county"] = geo_12.COUNTY.astype(int)
geo_12 = geo_12[["county", "UNIT_NAME", "VTD", "VTDI10", "UNITNUM", "geometry"]].sort_values(by = "VTD")

In [10]:
geo_12["precinct"] = geo_12["VTD"].str.replace(r'^([0-9]{,3}[A-FNSW]{,1})$', r'0-\1')\
                                  .str.replace(r'^([0-9]*)-([0-9]*)', r'0\1/0\2')\
                                  .str.replace(r'0([0-9]{2})', r'\1')

geo_12.loc[geo_12.county.isin([17, 33, 51, 55, 105, 109]), "precinct"] = geo_12.loc[geo_12.county.isin([17, 33, 51, 55, 105, 109]), "precinct"].str.replace(r'(0[0-5])/([0-9]{2}[AB]*)$', r'\1/0\2')
geo_12.loc[geo_12.county.isin([23]), "precinct"] = geo_12.loc[geo_12.county.isin([23]), "precinct"].str.replace(r"[EW]", "")
geo_12.loc[geo_12.county.isin([23]), "precinct"] = geo_12.loc[geo_12.county.isin([23]), "precinct"].str.replace(r"12[NS]", "12")


geo_12.loc[geo_12.county == 39, "precinct"] = geo_12.loc[geo_12.county == 39, "UNIT_NAME"].str.replace(r"Precinct ([0-9]{2})-([0-9]{4})", r"\1/\2")
geo_12.loc[geo_12.UNIT_NAME == "Precinct 2010A", "precinct"] = "02/2010A"
geo_12.loc[geo_12.UNIT_NAME == "Precinct 5010A", "precinct"] = "05/5010A"

geo_12.loc[geo_12.precinct.str.contains(r"1{,1}[89][AB]I"), "precinct"] = ("00/0" + geo_12.loc[geo_12.precinct.str.contains(r"1{,1}[89][AB]I"), "precinct"]).str.replace(r"/0([0-9]{2}[AB])", r"/\1")
geo_12.loc[(geo_12.county == 51) & geo_12.VTD.str.contains(r'[GHKLW]'), "precinct"] = geo_12.loc[(geo_12.county == 51) & geo_12.VTD.str.contains(r'[GHKLW]'), "VTD"].str.replace(r"^([0-9])", r"00\1").str.replace(r"^0*([0-9]{3})-([GIHKLW]{,2})", r"00/\2\1")
geo_12.loc[geo_12.county == 79, "precinct"] = geo_12.loc[geo_12.county == 79, "VTD"].str.replace(r"^([CNS])", r"00/\g<1>00").str.replace(r"^(00/[CNS])0*([0-9]{2})", r"\1\2")
geo_12.loc[geo_12.county == 85, "precinct"] = "00/" + geo_12.loc[geo_12.county == 85, "VTD"]

geo_12.loc[(geo_12.county == 3)   & geo_12.precinct.str.contains("05/01-A"), "precinct"] = "05/01A"
geo_12.loc[(geo_12.county == 15)  & geo_12.precinct.str.contains("02/05B"), "precinct"] = "02/05"
geo_12.loc[(geo_12.county == 103) & geo_12.precinct.str.contains(r'[ACFMPS]'), "precinct"] = "00/" + geo_12.loc[(geo_12.county == 103) & geo_12.precinct.str.contains(r'[ACFMPS]'), "precinct"]
geo_12.loc[(geo_12.county == 113) & geo_12.precinct.str.contains(r'-'), "precinct"] = ("00/" + geo_12.loc[(geo_12.county == 113) & geo_12.precinct.str.contains(r'-'), "precinct"]).str.replace("00/7A-1", "00/07A-1")
geo_12.loc[(geo_12.county == 113) & geo_12.precinct.str.contains("00/5A-1"), "precinct"] = "00/05A-1"
geo_12.loc[(geo_12.county == 119) & geo_12.precinct.str.contains(r'33I'), "precinct"] = "00/" + geo_12.loc[(geo_12.county == 119) & geo_12.precinct.str.contains(r'33I'), "precinct"]

geo_12.sort_values(by = ["county", "precinct"], inplace = True)

In [11]:
geo_votes_12 = geo_12.merge(votes_12, on = ["county", "precinct"], how = "right")[["county", "precinct", "D12", "R12", "geometry"]]
display(geo_votes_12.loc[(geo_votes_12.D12.isnull() | geo_votes_12.geometry.isnull()) & ~geo_votes_12.precinct.str.contains("Z")])
geo_votes_12.to_file(laf + "la_geo_votes_12.shp")

,county,precinct,D12,R12,geometry


### Gruesome merging for 2016

In [13]:
votes_16 = pd.read_csv(laf + "la_votes_2016.csv")

votes_16.loc[votes_16.county.isin([5, 7, 17, 33, 55, 57, 89, 97, 109]), "precinct"] = votes_16.loc[votes_16.county.isin([5, 7, 17, 33, 55, 57, 89, 97, 109]), "precinct"].str.replace(r" [ABCYZJKLM]", "")
votes_16.loc[votes_16.county.isin([15]), "precinct"] = votes_16.loc[votes_16.county.isin([15]), "precinct"].str.replace(r" -[1-2]$", r"")
votes_16.loc[votes_16.county.isin([25, 43, 75, 89, 91, 127]), "precinct"] = votes_16.loc[votes_16.county.isin([25, 43, 75, 89, 91, 127]), "precinct"].str.replace(r"/", r"/0")
votes_16.loc[votes_16.county.isin([29]), "precinct"] = votes_16.loc[votes_16.county.isin([29]), "precinct"].str.replace(r"00/", r"0").str.replace(r"-", r"/0")
votes_16.loc[votes_16.county.isin([67]), "precinct"] = votes_16.loc[votes_16.county.isin([67]), "precinct"].str.replace(r"00/([0-9]{2})-([12])", r"\1/0\2")
votes_16.loc[votes_16.county.isin([79]), "precinct"] = votes_16.loc[votes_16.county.isin([79]), "precinct"].str.replace(r" [AL][KZ]", "")

votes_16.loc[(votes_16.county == 3)   & (votes_16.precinct == "02/02A"), "precinct"] = "02/02-A"
votes_16.loc[(votes_16.county == 113) & (votes_16.precinct == "00/49B-2"), "precinct"] = "00/49B-1"
votes_16.loc[(votes_16.county == 121) & (votes_16.precinct == "00/02A"), "precinct"] = "00/02B"
votes_16.loc[(votes_16.county == 121) & (votes_16.precinct == "00/11B"), "precinct"] = "00/11A"

# votes_16.loc[votes_16.county == 105, "precinct"] = votes_16.loc[votes_16.county == 105, "precinct"].str.replace(r"0(28|40|42|45|70|72)A", r"0\1")


votes_16 = allocate_prov_abs_votes(votes_16, 2016)
votes_16 = votes_16.groupby(by = ["county", "precinct"]).sum().reset_index()

In [14]:
geo_16 = gpd.read_file(laf + "LA_2016.shp") # Had to re-save on the Mac to get field data.
geo_16["county"] = geo_16.COUNTY.astype(int)
geo_16 = geo_16[["county", "ID", "UNIT_NAME", "VTD", "VTDI10", "UNITNUM", "geometry"]].sort_values(by = "VTD")
geo_16["precinct"] = ""

In [15]:
geo_16["precinct"] = geo_16["VTD"].str.replace(r'^([0-9]{,3}[A-FNSW]{,1})$', r'0-\1')\
                                  .str.replace(r'^([0-9]*)-([0-9]*)', r'0\1/0\2')\
                                  .str.replace(r'0([0-9]{2})', r'\1')

geo_16.loc[geo_16.county.isin([17, 33, 51, 55, 105, 109]), "precinct"] = geo_16.loc[geo_16.county.isin([17, 33, 51, 55, 105, 109]), "precinct"].str.replace(r'(0[0-5])/([0-9]{2}[A]*)$', r'\1/0\2')

geo_16.loc[geo_16.county == 39, "precinct"] = geo_16.loc[geo_16.county == 39, "UNIT_NAME"].str.replace(r"Precinct ([0-9]{2})-([0-9]{4})", r"\1/\2")
geo_16.loc[geo_16.UNIT_NAME == "Precinct 2010A", "precinct"] = "02/2010A"

geo_16.loc[(geo_16.county == 49) & (geo_16.precinct == "19AI"), "precinct"] = "00/19AI"
geo_16.loc[(geo_16.county == 51) & geo_16.VTD.str.contains(r'[GHKLW]'), "precinct"] = geo_16.loc[(geo_16.county == 51) & geo_16.VTD.str.contains(r'[GHKLW]'), "VTD"].str.replace(r"^([0-9])", r"00\1").str.replace(r"^0*([0-9]{3})-([GIHKLW]{,2})", r"00/\2\1")
geo_16.loc[geo_16.county == 79, "precinct"] = geo_16.loc[geo_16.county == 79, "VTD"].str.replace(r"^([CNS])", r"00/\g<1>00").str.replace(r"^(00/[CNS])0*([0-9]{2})", r"\1\2")
geo_16.loc[geo_16.county == 85, "precinct"] = "00/" + geo_16.loc[geo_16.county == 85, "VTD"]
geo_16.loc[(geo_16.county == 87) & (geo_16.precinct == "00/58"), "precinct"] = "00/51" # This the sole non-match.
geo_16.loc[(geo_16.county == 103) & geo_16.precinct.str.contains(r'[ACFMPS]'), "precinct"] = "00/" + geo_16.loc[(geo_16.county == 103) & geo_16.precinct.str.contains(r'[ACFMPS]'), "precinct"]
geo_16.loc[(geo_16.county == 113) & geo_16.precinct.str.contains(r'-'), "precinct"] = ("00/" + geo_16.loc[(geo_16.county == 113) & geo_16.precinct.str.contains(r'-'), "precinct"]).str.replace("00/7A-1", "00/07A-1")

geo_16.sort_values(by = ["county", "precinct"], inplace = True)

In [16]:
geo_votes_16 = geo_16.merge(votes_16, on = ["county", "precinct"], how = "outer")[["county", "precinct", "D16", "R16", "geometry"]]
geo_votes_16.to_file(laf + "la_geo_votes_16.shp")

### For sanity, output these as pannable maps...

In [17]:
import matplotlib.pyplot as plt

def output_geojson(df, var, fname):

    fill = {}
    for ri, row in df.iterrows():
        color = [int(v*255) for v in plt.get_cmap("RdBu")(row[var])][:3]
        fill[ri] = "#{0:02X}{1:02X}{2:02X}".format(*color)

    df["fill"] = pd.Series(fill)
    df["stroke-width"] = 0
    df["fill-opacity"] = 0.15

    with open(fname, "w") as out: out.write(df.to_json())

geo_votes_12["DFr"] = geo_votes_12.D12 / (geo_votes_12.D12 + geo_votes_12.R12)
geo_votes_16["DFr"] = geo_votes_16.D16 / (geo_votes_16.D16 + geo_votes_16.R16)

# output_geojson(geo_votes_12, "DFr", "la_2012.geojson")
# output_geojson(geo_votes_16, "DFr", "la_2016.geojson")

### Now merge them to census tracts.

In [19]:
geo_votes_2012 = gpd.read_file(laf + "la_geo_votes_12.shp").to_crs(epsg = epsg)
geo_votes_2016 = gpd.read_file(laf + "la_geo_votes_16.shp").to_crs(epsg = epsg)

In [20]:
geo_votes_2012.head()

,D12,R12,county,geometry,precinct
0,100.189980,836.088559,1,"POLYGON ((913441.9005797947 199815.6807249119,...",01/01
1,133.179363,211.133475,1,"POLYGON ((910938.5424205648 192758.0687076475,...",01/02A
2,57.425964,336.607082,1,"POLYGON ((910699.4770064028 192761.775441711, ...",01/02B
3,37.876700,486.210230,1,"POLYGON ((910781.9804304764 193548.7171660828,...",01/03A
4,133.179363,231.643584,1,"POLYGON ((911929.1940590792 192999.6427604714,...",01/03B


In [21]:
query = """SELECT rn, ST_Centroid(ST_Transform(tr.geom, epsg)) geometry
           FROM census_tracts_2015 AS tr
           JOIN (SELECT state, county, tract, row_number() over 
                          (PARTITION BY state ORDER BY county, tract NULLS LAST) - 1 as rn
                 FROM census_tracts_2015) rn ON
                   tr.state  = rn.state  AND
                   tr.county = rn.county AND
                   tr.tract  = rn.tract
           JOIN states AS st ON st.fips = tr.state
           WHERE tr.state = {} ORDER BY rn;
           """

con = psycopg2.connect(database = "census", user = user, password = passwd,
                       host = "saxon.harris.uchicago.edu", port = 5432)

tr_rn = gpd.GeoDataFrame.from_postgis(query.format(fips), con, index_col = "rn",
                                      geom_col = "geometry", crs = from_epsg(epsg))

In [22]:
geo_votes_2012["rn"] = float('nan')
for pri, row in geo_votes_2012.iterrows():
    ctr = row.geometry.centroid
    distances = [(xi, pt.distance(ctr)) for xi, pt in enumerate(tr_rn.geometry)]
    match = min(distances, key=lambda item:item[1])
    geo_votes_2012.loc[pri, "rn"] = match[0]
    if (pri % 100) == 0: print(pri, end = " ")
print()
        
geo_votes_2016["rn"] = float('nan')
for pri, row in geo_votes_2016.iterrows():
    ctr = row.geometry.centroid
    distances = [(xi, pt.distance(ctr)) for xi, pt in enumerate(tr_rn.geometry)]
    match = min(distances, key=lambda item:item[1])
    geo_votes_2016.loc[pri, "rn"] = match[0]
    if (pri % 100) == 0: print(pri, end = " ")
        
geo_votes_2012.rn = geo_votes_2012.rn.astype(int)
geo_votes_2016.rn = geo_votes_2016.rn.astype(int)

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 

In [38]:
la_votes = pd.merge(geo_votes_2012[["rn", "D12", "R12"]], geo_votes_2016[["rn", "D16", "R16"]], 
                    on = "rn", how = "outer").fillna(0).groupby("rn").sum()
la_votes.to_csv("la_votes.csv")
la_votes.head(10)

,D12,R12,D16,R16
rn,,,,
0,7434,13986,7161,17619
1,315,4902,261,5565
2,244,1686,230,1956
3,928,9180,1624,17066
4,936,5571,849,6285
5,1539,2877,1374,3303
6,14440,31350,14014,29315
7,940,6190,828,7044
8,3468,4272,5187,6048
